# Разведывательный анализ данных

In [ ]:
%pip install numpy pandas matplotlib seaborn scikit-learn

[Скачать датасет]()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib as mplb
import matplotlib.pyplot as plt
import warnings
from scipy.stats import spearmanr, pearsonr

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

### Считываем данные 

In [ ]:
df = pd.read_csv(
    filepath_or_buffer="./invest_db.csv",
    delimiter=";",
    parse_dates=["date", "invest_utilization_dt"]
)

##### Общее кол-во строк

In [ ]:
df.shape

##### Рассмотрим структуру датафрейма

In [ ]:
df.head()

##### Основная информация о строках

In [ ]:
df.info()

##### Стат данные по столбцам

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.head(100)

##### Приводим типы данных

In [ ]:
# в данных дробные числа записаны через запятую
str_columns = []
for col in df.columns:
    if df[col].dtype == "object" and col not in ():
        str_columns.append(col)

str_columns

In [ ]:
for col in str_columns:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(',', '.', regex=False)
    )

In [ ]:
df.head(100)

In [ ]:
columns_to_convert_to_numeric = [
    'children_cnt',
    'monthly_income_amt',
    'in_payment_rub_amt',
    'out_payment_rub_amt',
    'uncovered_position_amt',
    'turnover_rus_bon_amt',
    'turnover_rus_sec_amt',
    'turnover_forts_amt',
    'turnover_cur_amt',
    'turnover_etf_amt',
    'turnover_fnd_amt',
    'turnover_opt_amt',
    'turnover_ore_prc_amt',
    'turnover_ore_sel_amt',
    'turnover_tracking_amt',
    'trade_order_rus_sec_cnt',
    'trade_order_rus_bon_cnt',
    'trade_order_forts_cnt',
    'trade_order_cur_cnt',
    'trade_order_etf_cnt',
    'trade_order_fnd_cnt',
    'trade_order_opt_cnt',
    'trade_order_ore_prc_cnt',
    'trade_order_ore_sel_cnt',
    'trade_order_tracking_cnt',
    'portf_total_amt',
    'portf_mex_amt',
    'portf_rus_sec_amt',
    'portf_rus_bon_amt',
    'initial_margin_amt',
    'portf_cur_amt',
    'portf_etf_amt',
    'portf_fnd_amt',
    'portf_opt_amt',
    'portf_ore_amt',
    'ccr_balance_amt',
    'cor_balance_amt',
    'lon_balance_amt',
    'posts',
    'comments',
    'reacts',
    'reads',
]

for col in columns_to_convert_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
df.head(100)

In [ ]:
df.education_level_cd.fillna("Undefined", inplace=True)
df.marital_status_cd.fillna("Undefined", inplace=True)
df.children_cnt.fillna(0, inplace=True)
df.monthly_income_amt.fillna(df.monthly_income_amt.median(), inplace=True)

df.ccr_balance_amt.fillna(df.ccr_balance_amt.mean(), inplace=True)
df.cor_balance_amt.fillna(df.ccr_balance_amt.mean(), inplace=True)
df.lon_balance_amt.fillna(df.ccr_balance_amt.mean(), inplace=True)

df.posts.fillna(0, inplace=True)
df.comments.fillna(0, inplace=True)
df.reacts.fillna(0, inplace=True)
df.reads.fillna(0, inplace=True)


In [ ]:
df.head(100)

In [ ]:
# как видим, все пропуски мы побороли
df.isna().sum()

In [ ]:
integer_cols = [
    'age',
    'children_cnt',
    'forts_flg', 
    'margin_status_flg', 
    'qualified_investor_flg',
    'posts',
    'comments',
    'reacts', 
    'reads'
    ]
for col in integer_cols:
    df[col] = df[col].astype(pd.Int64Dtype())
df.dtypes

In [ ]:
df.describe()

##### Преобразуем категориальные переменные

In [ ]:
df_edu_encoded = pd.get_dummies(df['education_level_cd'], prefix='edu')
df_marital_encoded = pd.get_dummies(df['marital_status_cd'], prefix='marital')

df = pd.concat([df, df_edu_encoded, df_marital_encoded], axis=1)

In [ ]:
df.columns

In [ ]:
# посчитаем приблизительное общее кол-во сделок
df["trade_total_cnt"] = df.trade_order_rus_sec_cnt + df.trade_order_rus_bon_cnt + df.trade_order_forts_cnt + df.trade_order_cur_cnt + df.trade_order_etf_cnt + df.trade_order_fnd_cnt + df.trade_order_opt_cnt + df.trade_order_ore_prc_cnt

In [ ]:
turnover_cols = [
    'turnover_rus_bon_amt', 'turnover_rus_sec_amt', 'turnover_forts_amt',
    'turnover_cur_amt', 'turnover_etf_amt', 'turnover_fnd_amt',
    'turnover_opt_amt', 'turnover_ore_prc_amt', 'turnover_ore_sel_amt',
]
df['turnover_total_amt'] = df[turnover_cols].sum(axis=1)

In [ ]:
def normalize(data:pd.Series):
    if data.dtype is object:
        raise ValueError("Invalid data format given")
    data = (data - data.min())/(data.max()-data.min())
    return data

# создадим синтетический признак - коэфициент приверженности к риску
df['risk_weighted_index'] = (
    (df['turnover_forts_amt'] * 3) + 
    (df['turnover_opt_amt'] * 3) + 
    (df['turnover_rus_sec_amt'] * 2) + 
    (df['turnover_rus_bon_amt'] * 1) + 
    (df['turnover_etf_amt'] * 1) + 
    (df['turnover_fnd_amt'] * 1) + 
    (df['turnover_ore_prc_amt'] * 1) +
    (df['turnover_ore_sel_amt'] * 1) +
    (df['forts_flg'] * 5000) +
    (df['margin_status_flg'] * 5000)
)

In [ ]:
# создадим отдельные колонки для риск-инструментов и консервативных инструментов


# Оборот с консервативными инструментами
df['turnover_conserv_amt'] = (
    df['turnover_rus_bon_amt'] + 
    df['turnover_etf_amt'] + 
    df['turnover_fnd_amt'] + 
    df['turnover_ore_prc_amt'] + 
    df['turnover_ore_sel_amt']
)

# Оборот с рисковыми инструментами
df['turnover_risk_amt'] = (
    df['turnover_rus_sec_amt'] + 
    df['turnover_forts_amt'] + 
    df['turnover_opt_amt']
)

# Количество сделок с консервативными инструментами
df['trade_conserv_cnt'] = (
    df['trade_order_rus_bon_cnt'] + 
    df['trade_order_etf_cnt'] + 
    df['trade_order_fnd_cnt'] + 
    df['trade_order_ore_prc_cnt'] + 
    df['trade_order_ore_sel_cnt']
)

# Количество сделок с рисковыми инструментами
df['trade_risk_cnt'] = (
    df['trade_order_rus_sec_cnt'] + 
    df['trade_order_forts_cnt'] + 
    df['trade_order_opt_cnt']
)

#  Объем консервативных инструментов в портфеле
df['portf_conserv_amt'] = (
    df['portf_rus_bon_amt'] + 
    df['portf_etf_amt'] + 
    df['portf_fnd_amt'] + 
    df['portf_ore_amt']
)

# Объем рисковых инструментов в портфеле
df['portf_risk_amt'] = (
    df['portf_rus_sec_amt'] + 
    df['portf_opt_amt']
)


### Непосредственно анализ данных

In [ ]:
plt.figure(figsize=(45, 35), dpi=300)
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Корреляционная матрица признаков")
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(35, 45))
sns.clustermap(df.corr(numeric_only=True), annot=True, fmt=".2f", cmap='coolwarm',figsize=(40, 30))
plt.show()
plt.close()

### Основные наблюдения

##### Положительная корреляция
1. статус квал инвестора, флаг маржинальной торговли и флаг подключения срочного рынка попарно положительно коррелируют между собой
2. Cумма ввода денежных средств за день положительно коррелирует с суммой вывода средств за день
3. Оборот фьючерсов положительно коррелирует с оборотом российских активов (слабая корреляция)
4. Оборот покупки ETF имеет положительную корреляцию с вводом средств, но не имеют таковой с выводом
5. Оборот фондов Т-Капитала имеет положительную корреляцию с вводом и выводом средств
6. Объем денежных активов в портфеле имеет очень сильную корелляцию с суммой по непокрытым позициям
7. Сумма гарантийного обеспечения имеет среднюю корелляцию с оборотом российских активов
8. Оборот по фьючерсам имеет положительную корреляцию с суммой гарантийного обеспечения (слабая)
9. Объем купленных драгоценных металлов кореллирует с оборотом фондов Т-капитала
10. Объем проданных драгоценных металлов кореллирует с оборотом фондов Т-капитала
11. Количество сделок при использовании автоследования кореллирует с кол-вом сделок в российском сегменте и сегменте ETF
12. Объем портфеля клиента на Московской бирже кореллирует с общим объемом портфеля
13. Кол-во российских активов (акций, ETF) очень сильно кореллирует с объемом портфеля
14. Объем денег на дебетвых и сберегательных счетах кореллирует с полным объемом портфеля
15. активность в соц-сети (предположительно в Пульсе) кореллирует между собой
16. Очень сильная корелляция между полным оборотом и оборотом фондов Т-капитала

##### Отрицательная корреляция
1. Маржа безопасности отрицательно коррелирует с непокрытыми активами
2. Оборот в российском сегменте имеет обратную корелляцию с непокрытыми активами
3. Объем заемных средств имеет слабую отрицательную корелляцию с объемом купленных ETF
4. Текущая сумма потфеля отрицательно кореллирует с оборотом российских активов (слабо)

### Основные гипотезы

1. Возраст не кореллирует ни с какими параметрами
2. Возраст кореллирует с риск-профилем ивестора
3. Образование связано с инвестированием в рисковые инструменты
4. Образование связано с риск-профилем инвестора
5. Объем портфеля связан с инвестированием в деривативы
6. Объем портфеля связан с активностью в Пульсе
7. Пол связан с выбором фин инструментов
8. Семейное положение связано с выбором фин инструментов
9. Возраст связан с использованием автоследования
10. Семейный статус влияет на инвестиционное поведение и на выбор консервативных или рискованных активов [x]
11. Крупные инвесторы чаще используют автоследование
12. Наличие непокрытой позиции не всегда означает, что у клиента не хватает средств
13. Объём заёмных средств связан с активностью на срочном рынке и маржинальной торговлей
14. Активность в Пульсе связана с оборотом в фьючерсах и опционах
15. Большие объёмы портфеля связаны с активностью на кредитных и дебетовых счетах 
16. Объём заёмных средств связан с более высокой активностью на рынке, а не с размером портфеля
17. Инвестирование в драгоценные металлы является отдельной стратегией, независимой от других видов инвестиций
18. Инвесторы с высоким уровнем образования имеют более крупные портфели и чаще торгуют на срочном рынке 
19. Фонды Т-капитала используются для трейдинга зачастую


### Целевые метрики
- portf_total_amt
- in_payment_rub_amt
- turnover_total_amt

```python
target_metrics = ["portf_total_amt", "in_payment_rub_amt", "turnover_total_amt"]
```

### Гипотеза 1-2
### Возраст не кореллирует ни с какими параметрами

In [ ]:
sns.regplot(data=df, x='age', y='portf_total_amt', scatter_kws={"s": 10}, line_kws={"color": "red"})
plt.title('Возраст и сумма портфеля')
plt.show()
plt.close()

bins = list(range(14, 100, 2))

df['age_group'] = pd.cut(df['age'], bins=bins, right=False)


plt.figure(figsize=(16, 8))
sns.barplot(x='age_group', y='portf_total_amt', data=df)
plt.xticks(rotation=45, ha='right')
plt.yscale('log')
plt.title('Средний объем портфеля по возрастным группам (логарифмическая шкала)')
plt.xlabel('Возрастная группа')
plt.ylabel('Средний объем портфеля, руб. (лог. шкала)')
plt.tight_layout()
plt.show()
plt.close()


видна небольшая просадка в возрасте 18 - 22, что вполне логично, так как мало кто в таком возрасте получает стабильный заработок

In [ ]:
sns.regplot(data=df, x='age', y='turnover_total_amt', scatter_kws={"s": 10}, line_kws={"color": "red"})
plt.show()
plt.close()

bins = list(range(14, 100, 2))

df['age_group'] = pd.cut(df['age'], bins=bins, right=False)


plt.figure(figsize=(16, 8))
sns.barplot(x='age_group', y='turnover_total_amt', data=df)
plt.xticks(rotation=45, ha='right')
plt.title("Распределение оборота по возрастным группам")
plt.xlabel("Возраст")
plt.ylabel("Оборот")
plt.tight_layout()
plt.show()
plt.close()


plt.figure(figsize=(16, 8))
sns.scatterplot(x='age', y='turnover_total_amt', data=df)
plt.tight_layout()
plt.show()
plt.close()

- отчетливо видно, что в возрасте 28-30 лет у людей выше оборот, это может быть связано с появлением денег для трейдинга или рисковых сделок
- менее значительный пик видно в бакете 52-54 года

In [ ]:
sns.regplot(data=df, x='age', y='risk_weighted_index', scatter_kws={"s": 10}, line_kws={"color": "red"})
plt.show()
plt.close()

bins = list(range(14, 100, 2))

df['age_group'] = pd.cut(df['age'], bins=bins, right=False)


plt.figure(figsize=(16, 8))
sns.barplot(x='age_group', y='risk_weighted_index', data=df)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.scatterplot(x='age', y='risk_weighted_index', data=df)
plt.tight_layout()
plt.show()
plt.close()

- Опять наблюдается тенденция повышения толерантности к риску в возрасте 28-30 лет
- Также виден скачек в бакете 52 - 54 года

In [ ]:
sns.regplot(data=df, x='age', y='in_payment_rub_amt', scatter_kws={"s": 10}, line_kws={"color": "red"})
plt.show()

bins = list(range(14, 100, 2))

df['age_group'] = pd.cut(df['age'], bins=bins, right=False)


plt.figure(figsize=(16, 8))
sns.barplot(x='age_group', y='in_payment_rub_amt', data=df)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.scatterplot(x='age', y='in_payment_rub_amt', data=df)
plt.tight_layout()
plt.show()

### ВЫВОД
исходя из предыдущих визуализций можно сделать выводы, что: 
- в возрасте 28-30 и 54-56 лет люди более склонны к трейдингу и риску
- в возрасте 18-22 инвесторы не используют большие денежные средства

### Гипотеза 3-4
3. Образование связано с инвестированием в рисковые активы
4. Образование связано с риск-профилем инвестора

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='risk_weighted_index', data=df)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.scatterplot(x='education_level_cd', y='risk_weighted_index', data=df)
plt.tight_layout()
plt.show()

видно, что люди с хотя бы одним высшим более склонны к риску, давайте посмотрим теперь на более тонкие сравнения

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='turnover_risk_amt', data=df)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='trade_risk_cnt', data=df)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='portf_risk_amt', data=df)
plt.xticks(rotation=45, ha='right')
plt.axhline(color="black")
plt.title("Зависимость суммы рисковых активов от образования")
plt.xlabel("Образование")
plt.ylabel("Сумма рисковых активов в портфеле")
plt.tight_layout()
plt.show()





люди с двумя высшими явно впереди, но вопросы вызывают отрицательные значения у людей со школьным образованием

In [ ]:
financial_cols = [
    'portf_total_amt', 'portf_mex_amt', 'portf_rus_sec_amt', 
    'portf_rus_bon_amt', 'portf_cur_amt', 'portf_etf_amt', 
    'portf_fnd_amt', 'portf_opt_amt', 'portf_ore_amt', 
    'ccr_balance_amt', 'cor_balance_amt', 'lon_balance_amt',
    'in_payment_rub_amt', 'out_payment_rub_amt', 'uncovered_position_amt',
    'turnover_rus_bon_amt', 'turnover_rus_sec_amt', 'turnover_forts_amt',
    'turnover_cur_amt', 'turnover_etf_amt', 'turnover_fnd_amt',
    'turnover_opt_amt', 'turnover_ore_prc_amt', 'turnover_ore_sel_amt',
    'turnover_tracking_amt'
]


negative_counts = {}

for col in financial_cols:
    if col in df.columns:
        negative_counts[col] = (df[col] < 0).sum()


for col, count in negative_counts.items():
    if count > 0:
        print(f"В колонке '{col}' найдено {count} отрицательных значений.")


total_negatives = sum(negative_counts.values())
print(f"\nВсего найдено {total_negatives} отрицательных значений во всех финансовых колонках.")

значений много, значит это не выбросы. вероятно, люди потеряли много денег используя маржинальную торговлю

Проверим теперь по консервативным инструментам

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='turnover_conserv_amt', data=df)
plt.xticks(rotation=45, ha='right')
plt.title("Зависимость оборота консервативных активов от образования")
plt.xlabel("Образование")
plt.ylabel("Оборот консервативных активов в портфеле")
plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='trade_conserv_cnt', data=df)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()


plt.figure(figsize=(16, 8))
sns.barplot(x='education_level_cd', y='portf_conserv_amt', data=df)
plt.xticks(rotation=45, ha='right')
plt.title("Зависимость суммы консервативных активов от образования")
plt.xlabel("Образование")
plt.ylabel("Сумма консервативных активов в портфеле")
plt.tight_layout()
plt.show()

### ВЫВОД

по обороту с консервативными инструментами явно лидируют люди с высшим образованием

явно заметна просадка со стороны людей без высшего образования

стоит заметить, что у людей с высшим образованием оборот по гораздо выше чем у остальных категорий по всем инструментам

также, внимание стоит уделить тому факту, что риску также очень склонны люди без высшего образования, это может быть связано с попыткой заработать без какой-либо стратегии инвестирования, попытками маркет-тайминга или трейдинга

ИТОГО: больше всего склонны к риску люди с одним высшим или, наоборот, со средним образованием


### Гипотеза 5-6
5. Объем портфеля связан с инвестированием в деривативы
6. Объем портфеля связан с активностью в Пульсе

In [ ]:
# введем нормированный по 100 бальной шкале коэфициент активности в пульсе

df["activity"] = (3*df.posts + 2*df.comments + 1.5*df.reacts + df.reads).astype(pd.Float64Dtype())
df["activity"] = (df["activity"] - df["activity"].min())/(df["activity"].max() - df["activity"].min())*100


In [ ]:
df.columns

In [ ]:


features_to_plot = [
       'trade_total_cnt',
       'turnover_total_amt',
       'risk_weighted_index',
       'turnover_conserv_amt',
       'turnover_risk_amt',
       'trade_conserv_cnt',
       'trade_risk_cnt',
       'portf_conserv_amt',
       'portf_risk_amt'
]


for feature in features_to_plot:
       g = sns.JointGrid(x='portf_total_amt', y=feature, data=df, height=10)
       g.plot_joint(sns.scatterplot, alpha=0.3)
       
       sns.kdeplot(x=df['portf_total_amt'], ax=g.ax_marg_x)
       sns.kdeplot(y=df[feature], ax=g.ax_marg_y)
       

       if 'amt' in feature or 'index' in feature:
              g.ax_joint.set_yscale('log')
       g.ax_joint.set_xscale('log')
       
       g.set_axis_labels('Объем портфеля', feature)

       plt.suptitle(f'Зависимость {feature} от объема портфеля', y=1.02)
       
       plt.show()
       plt.close()



Из действительно ценных фактов можно подчеркнуть, что: 
- объем портфеля имеет положительную корелляцию с оборотом, что логично
- объем имеет довольно сильную корелляцию с объемом как рисковых, так и консервативных активов (грубо говоря, этот факт нам ничего не дает)

#### Теперь рассмотрим связь с активностью в пульсе

In [ ]:

target_metrics = ["portf_total_amt", "in_payment_rub_amt", "turnover_total_amt"]




for metric in target_metrics:
    plt.figure(figsize=(16, 8))
    sns.regplot(x=metric, y="activity", data=df, scatter_kws={'alpha': 0.3})
    

    plt.title(f'Зависимость активности от {metric}')
    plt.xlabel(f'{metric}')
    plt.ylabel('Активность в Пульсе')
    
    plt.tight_layout()

    plt.show()
    plt.close()


##### ВЫВОД
- объем портфеля имеет положительную корелляцию с оборотом 
- зависимость оборота и размера портфеля от активности в пульсе имеет вид экспоненциального распределения

### Гипотеза 7
- Связь пола с инструментами инвестирования
Невозможно проверить, так как пол не представлен в данных

### Гипотеза 8-10
1. Семейное положение связано с выбором фин инструментов
2. Возраст связан с использованием автоследования
3.  Семейный статус влияет на инвестиционное поведение и на выбор консервативных или рискованных активов

In [ ]:
criteria = [
    'trade_total_cnt',
    'turnover_total_amt',
    'risk_weighted_index',
    'turnover_conserv_amt',
    'turnover_risk_amt',
    'trade_conserv_cnt',
    'trade_risk_cnt',
    'portf_conserv_amt',
    'portf_risk_amt',
    "portf_total_amt",
    "in_payment_rub_amt",
    "turnover_total_amt",
    "turnover_tracking_amt"
]

for crit in criteria:
    plt.figure(figsize=(16, 8))
    sns.barplot(x='marital_status_cd', y=crit, data=df)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    plt.close()

Заметим, что:
- Больше всего оборот средств у вдовцов и женатых
- У вдовцов больше всего портфель
- У вдовцов больше всех рисковых активов
- У женатых больше всего оборот консервативных инструментов

ВЫВОДЫ
- Вдовцы и женатые - 2 самые активные группы пользователей
- Вдовцы скорее выберут рисковые активы
- Женатые скорее выберут консервативные активы

### Гипотеза 11
11. Крупные инвесторы чаще используют автоследование

In [ ]:
df.columns

In [ ]:
target_metrics = ["portf_total_amt", "in_payment_rub_amt", "turnover_total_amt", "risk_weighted_index"]
for metric in target_metrics:
    plt.figure(figsize=(16, 8))
    sns.regplot(x=metric, y="turnover_tracking_amt", data=df, scatter_kws={'alpha': 0.3})
    

    plt.title(f'Зависимость turnover_tracking_amt от {metric}')
    plt.xlabel(f'{metric}')
    plt.ylabel('turnover_tracking_amt')
    
    plt.tight_layout()

    plt.show()
    plt.close()


Зависимость очень похожа на экспоненциальное распределение

##### ВЫВОД
- Чем крупнее инвестор, тем реже он использует автоследование

### Гипотеза 12
1.  Наличие непокрытой позиции не всегда означает, что у клиента не хватает средств

In [ ]:
df.uncovered_position_amt.describe()

In [ ]:
target_metrics = ["portf_total_amt", "in_payment_rub_amt", "turnover_total_amt", "risk_weighted_index"]
for metric in target_metrics:
    plt.figure(figsize=(16, 8))
    sns.regplot(x=metric, y="uncovered_position_amt", data=df, scatter_kws={'alpha': 0.3})


    plt.title(f'Зависимость uncovered_position_amt от {metric}')
    plt.xlabel(f'{metric}')
    plt.ylabel('uncovered_position_amt')
    
    plt.tight_layout()

    plt.show()
    plt.close()

### ВЫВОД
- из графиков видно, что маржинальная торговля обратно пропорциональна сумме поплнений и баллансу по счету, то есть более крупные инвесторы не используют маржинальную торговлю

### Гипотеза 13
1.  Объём заёмных средств связан с активностью на срочном рынке и маржинальной торговлей


### ВЫВОД
- Это подтверждается из предыдущего пункта исследования

### Гипотеза 14
1.  Активность в Пульсе связана с оборотом в фьючерсах и опционах

In [ ]:
target_metrics = ["portf_total_amt", "in_payment_rub_amt", "turnover_total_amt", "risk_weighted_index", "turnover_opt_amt", "turnover_forts_amt"]
for metric in target_metrics:
    plt.figure(figsize=(16, 8))
    sns.regplot(x=metric, y="activity", data=df, scatter_kws={'alpha': 0.3})


    plt.title(f'Зависимость activity от {metric}')
    plt.xlabel(f'{metric}')
    plt.ylabel('activity')
    
    plt.tight_layout()

    plt.show()
    plt.close()

### ВЫВОД
- активность в пульсе обратно пропорциональна оборотам
- чем активнее инвестор, тем с большей вероятностью он новичок

### Гипотеза 15 - 16
1.  Большие объёмы портфеля связаны с активностью на кредитных и дебетовых счетах
2.  Объём заёмных средств связан с более высокой активностью на рынке, а не с размером портфеля

In [ ]:

target_metrics = [
    "portf_total_amt", 
    "in_payment_rub_amt", 
    "turnover_total_amt", 
    "risk_weighted_index", 
    "turnover_opt_amt", 
    "turnover_forts_amt"]

ballance_metrics = ["ccr_balance_amt", "cor_balance_amt", "lon_balance_amt"]


for targ in target_metrics:
    for bal in ballance_metrics:
        plt.figure(figsize=(16, 8))

        x = df[targ]
        y = df[bal]
        mask = x.notna() & y.notna()
        x_masked = x[mask]
        y_masked = y[mask]


        spearman_corr, spearman_p = spearmanr(x_masked, y_masked)


        pearson_corr, pearson_p = pearsonr(x_masked, y_masked)

        sns.regplot(x=x_masked, y=y_masked, scatter_kws={'alpha': 0.3})

        plt.title(
            f'{bal} vs {targ}\n'
            f'Spearman ρ = {spearman_corr:.2f} (p = {spearman_p:.3f})\n'
            f'Pearson r = {pearson_corr:.2f} (p = {pearson_p:.3f})'
        )

        plt.xlabel(targ)
        plt.ylabel(bal)
        plt.tight_layout()
        plt.show()
        plt.close()

### ВЫВОД
- Существует корелляция между баллансом на дебетовых и сберегательных счетах и размером портфеля
- Долг на кредитных счетах прямо пропорционален коэфициенту риска а также сумме пополнений
- Сумма портфеля не связана с суммой на кредитных счетах

### Гипотеза 17
1.  Инвестирование в драгоценные металлы является предполагает консервативную стратегию

In [ ]:
gold = ["trade_order_ore_prc_cnt", "turnover_ore_prc_amt", "portf_ore_amt"]
categ = ["education_level_cd", "marital_status_cd"]
cons = [
    "risk_weighted_index", 
    "turnover_opt_amt", 
    "turnover_forts_amt"
]
for g in gold:
    for cat in categ:
        plt.figure(figsize=(16, 8))
        sns.barplot(x=cat, y=g, data=df)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
        plt.close()


for g in gold:
    for cat in cons:
        plt.figure(figsize=(16, 8))
        sns.regplot(x=g, y=cat, data=df)
        plt.tight_layout()
        plt.show()
        plt.close()

### ВЫВОД
- люди с двумя высшими и ученой степенью более склонны с инвестициям в золото
- золото является прерогативой консервативных инвесторов

### Гипотеза 18
1.  Инвесторы с высоким уровнем образования имеют более крупные портфели и чаще торгуют на срочном рынке 


In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(x="education_level_cd", y="trade_order_forts_cnt", data=df)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
plt.close()



plt.figure(figsize=(16, 8))
sns.barplot(x="education_level_cd", y="portf_total_amt", data=df)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
plt.close()


plt.figure(figsize=(16, 8))
sns.barplot(x="education_level_cd", y="in_payment_rub_amt", data=df)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
plt.close()

### ВЫВОД
- Люди со слабым обрзованием имеют меньшие портфели
- С деривативами в большинстве работают люди с 2 высшими образованиями

### Гипотеза 19
1.  Фонды Т-капитала используются для трейдинга зачастую

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(16, 8))
sns.regplot(x="turnover_fnd_amt", y="portf_risk_amt", data=df)
plt.tight_layout()
plt.show()
plt.close()

plt.figure(figsize=(16, 8))
sns.regplot(x="turnover_fnd_amt", y="turnover_risk_amt", data=df)
plt.tight_layout()
plt.show()
plt.close()


plt.figure(figsize=(16, 8))
sns.regplot(x="turnover_fnd_amt", y="risk_weighted_index", data=df)
plt.tight_layout()
plt.show()
plt.close()

### ВЫВОД
- Оборот фондами Т-Банка меет очень сильную корелляцию с взвешенным индексом риска

### Дополнительное исследование

Попробуем отобрать из всех записей тех, кто больше всего подвержен риску, а также тех, кто меньше всего подвержен риску, а затем изучим их портреты

In [ ]:
df_risk = df[(df.forts_flg==1) | (df.margin_status_flg==1) | (df.uncovered_position_amt<0) | (df.turnover_forts_amt != 0) | (df.turnover_opt_amt != 0)]

bonds_quant = df.turnover_rus_bon_amt.quantile(0.25)
etf_quant = df.turnover_etf_amt.quantile(0.25)
gold_quant = df.turnover_ore_prc_amt.quantile(0.25)
df_conserv = df[
    ~(
        (df.forts_flg==1) | (df.margin_status_flg==1) | (df.uncovered_position_amt<0) | (df.turnover_forts_amt != 0) | (df.turnover_opt_amt != 0)
    ) 
    |
    (df.turnover_rus_bon_amt > bonds_quant) | (df.turnover_etf_amt > etf_quant) | (df.turnover_ore_prc_amt > gold_quant)
]

plt.figure(figsize=(45, 35))
sns.heatmap(df_risk.corr(numeric_only=True), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Корреляционная матрица для агрессивных инвесторов")
plt.show()
plt.close()



categorical_cols = ["education_level_cd", "marital_status_cd"]
titles = [
    "Сегменты по образованию среди агрессивных инвесторов",
    "Сегменты по семейному положению среди агрессивных инвесторов",
]

for col, title in zip(categorical_cols, titles):
    counts = df_risk[col].value_counts(normalize=True).sort_values(ascending=True)

    plt.figure(figsize=(10, 6))
    sns.barplot(x=counts.values * 100, y=counts.index, palette="viridis")

    for i, (v, name) in enumerate(zip(counts.values, counts.index)):
        plt.text(v * 100 + 0.5, i, f"{v*100:.1f}%", va='center', fontsize=12)

    plt.title(title, fontsize=16)
    plt.xlabel("Доля, %", fontsize=14)
    plt.ylabel("")
    plt.grid(axis='x', linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()
    plt.close()


plt.figure(figsize=(10, 6))
sns.histplot(data=df_risk, x="children_cnt", discrete=True)
plt.title("Распределение по количеству детей", fontsize=16)
plt.xlabel("Количество детей", fontsize=14)
plt.ylabel("Частота", fontsize=14)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()
plt.close()


plt.figure(figsize=(20, 16))
sns.regplot(x=df_risk.portf_total_amt, y='monthly_income_amt', data=df_risk)
plt.title("Сегменты по объему портфеля среди агрессивных инвесторов")
plt.show()
plt.close()

plt.figure(figsize=(20, 16))
sns.histplot(data=df_risk, x='monthly_income_amt', bins='auto', kde=True)
plt.title("Распределение месячного дохода агррессивных инесторов")
plt.show()
plt.close()


In [ ]:
plt.figure(figsize=(45, 35))
sns.heatmap(df_conserv.corr(numeric_only=True), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Корреляционная матрица для консервативных инвесторов")
plt.show()
plt.close()



categorical_cols = ["education_level_cd", "marital_status_cd"]
titles = [
    "Сегменты по образованию среди консервативных инвесторов",
    "Сегменты по семейному положению среди консервативных инвесторов",
]

for col, title in zip(categorical_cols, titles):
    counts = df_conserv[~df_conserv[col].isna()][col].value_counts(normalize=True).sort_values(ascending=True)

    plt.figure(figsize=(10, 6))
    sns.barplot(x=counts.values * 100, y=counts.index, palette="viridis")

    for i, (v, name) in enumerate(zip(counts.values, counts.index)):
        plt.text(v * 100 + 0.5, i, f"{v*100:.1f}%", va='center', fontsize=12)

    plt.title(title, fontsize=16)
    plt.xlabel("Доля, %", fontsize=14)
    plt.ylabel("")
    plt.grid(axis='x', linestyle='--', alpha=0.4)
    plt.tight_layout()
    plt.show()
    plt.close()


plt.figure(figsize=(10, 6))
sns.histplot(data=df_conserv, x="children_cnt", discrete=True)
plt.title("Распределение по количеству детей", fontsize=16)
plt.xlabel("Количество детей", fontsize=14)
plt.ylabel("Частота", fontsize=14)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()
plt.close()


plt.figure(figsize=(20, 16))
sns.regplot(x=df_conserv.portf_total_amt, y='monthly_income_amt', data=df_conserv)
plt.title("Сегменты по объему портфеля среди консервативных инвесторов")
plt.show()
plt.close()

plt.figure(figsize=(20, 16))
sns.histplot(data=df_conserv, x='monthly_income_amt', bins='auto', kde=True)
plt.title("Распределение месячного дохода консервативных инесторов")
plt.show()
plt.close()

In [ ]:
q_risk = df_risk['monthly_income_amt'].quantile(0.95)
q_conserv = df_conserv['monthly_income_amt'].quantile(0.95)
max_val = max(q_risk, q_conserv)

df_risk_filtered = df_risk[(df_risk['monthly_income_amt'] > 0) & (df_risk['monthly_income_amt'] <= max_val)]
df_conserv_filtered = df_conserv[(df_conserv['monthly_income_amt'] > 0) & (df_conserv['monthly_income_amt'] <= max_val)]


plt.figure(figsize=(14, 8), dpi=150)

sns.histplot(
    data=df_risk_filtered,
    x='monthly_income_amt',
    bins='auto',
    kde=True,
    label="Агрессивные инвесторы",
    stat='density',
    common_norm=False,
    color='skyblue'
)

sns.histplot(
    data=df_conserv_filtered,
    x='monthly_income_amt',
    bins='auto',
    kde=True,
    label="Консервативные инвесторы",
    stat='density',
    common_norm=False,
    color='salmon'
)

plt.title("Распределение ежемесячного дохода инвесторов", fontsize=16)
plt.xlabel("Ежемесячный доход", fontsize=14)
plt.ylabel("Плотность", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()
plt.close()

ВЫВОДЫ
- среди агрессивных инвесторов больше неженатых и разведенных, также больше доля бездетных
- среди консервативных инвесторов больше доля женатых, больше доля людей с высшим образованием, больше доля тех, у кого есть дети
- распределения месячных доходов очень схожи, но на удивление, пик в моде у консервативных инвесторов выше

## Итоговые подтвердившиеся гипотезы

- в возрасте 28-30 и 54-56 лет люди более склонны к трейдингу и риску
- в возрасте 18-22 инвесторы не используют большие денежные средства
- больше всего склонны к риску люди с одним высшим или, наоборот, со средним образованием
- объем портфеля имеет положительную корелляцию с оборотом 
- зависимость оборота и размера портфеля от активности в пульсе имеет вид экспоненциального распределения
- Вдовцы и женатые - 2 самые активные группы пользователей
- Вдовцы скорее выберут рисковые активы
- Женатые скорее выберут консервативные активы
- Чем крупнее инвестор, тем реже он использует автоследование
- маржинальная торговля обратно пропорциональна сумме поплнений и баллансу по счету, то есть более крупные инвесторы не используют маржинальную торговлю
- активность в пульсе обратно пропорциональна оборотам
- чем активнее инвестор, тем с большей вероятностью он новичок
- Существует корелляция между баллансом на дебетовых и сберегательных счетах и размером портфеля
- Долг на кредитных счетах прямо пропорционален коэфициенту риска а также сумме пополнений
- Сумма портфеля не связана с суммой на кредитных счетах
- люди с двумя высшими и ученой степенью более склонны с инвестициям в золото
- золото является прерогативой консервативных инвесторов
- Оборот фондами Т-Банка меет очень сильную корелляцию с взвешенным индексом риска
- среди агрессивных инвесторов больше неженатых и разведенных, также больше доля бездетных
- среди консервативных инвесторов больше доля женатых, больше доля людей с высшим образованием, больше доля тех, у кого есть дети
- распределения месячных доходов очень схожи, но на удивление, пик в моде у консервативных инвесторов выше


##### Выводы

- Возраст и семейное положение влияют на инвестиционное поведение
- Есть два возрастных пика активности и риска
- Женатые и вдовцы — разные поведенческие сегменты: вдовцы — более агрессивные, вероятно, из-за меньшего уровня обязательств или возрастного статуса


- Баланс портфеля — индикатор зрелости и опыта: крупные инвесторы торгуют активно, но осторожно, избегая риска и автоследования
- Маржинальная торговля и автоследование — инструменты скорее для новичков или агрессивных малых инвесторов


- Образование влияет на тип инвестиционного поведения
- Люди с продвинутым образованием склонны к инвестициям в золото


- Пульс — инструмент вовлечения новых, но не крупных инвесторов
- Часто активность в соц.сети является сигналом начинающего или мелкого инвестора


- Использование кредитных средств не говорит о размере капитала — скорее, о риск-профиле и нестабильности


- Есть фондовые инструменты, чётко связанные с поведением рисковых инвесторов

- Среди агрессивных инвесторов больше неженатых и разведенных, также больше доля бездетных
- Среди консервативных инвесторов больше доля женатых, больше доля людей с высшим образованием, больше доля тех, у кого есть дети
- Распределения месячных доходов очень схожи, но на удивление, пик в моде у консервативных инвесторов выше



#### Данные для оценки доли рынка

In [ ]:
df.portf_total_amt.sum()

In [ ]:
df.shape

In [ ]:
df.portf_total_amt.mean()

In [ ]:
df.portf_total_amt.median()

In [ ]:

plt.figure(figsize=(16, 5), dpi=300)

sns.histplot(
    data=df,
    x='portf_total_amt',
    bins=200,
    stat='density',
    common_norm=False,
    color='salmon'
)

plt.title("Распределение суммы портфелей инвесторов", fontsize=16)
plt.xlabel("Сумма портфеля, ₽", fontsize=14)
plt.ylabel("Плотность", fontsize=14)
plt.yscale("log")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
plt.close()


plt.figure(figsize=(14,6), dpi=150)

sns.ecdfplot(data=df, x='portf_total_amt')
plt.xscale('log')
plt.title("Кумулятивное распределение портфелей инвесторов", fontsize=16)
plt.xlabel("Сумма портфеля, ₽ (лог)", fontsize=14)
plt.ylabel("Доля инвесторов", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6), dpi=150)


sns.histplot(
    data=df,
    x='portf_total_amt',
    bins="auto",
    stat='density',
    common_norm=False,
    color='salmon',
    alpha=0.6,
    label='Плотность портфелей',
    ax=ax1
)
ax1.set_xlabel("Сумма портфеля, ₽", fontsize=14)
ax1.set_ylabel("Плотность", fontsize=14)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.grid(axis='y', linestyle='--', alpha=0.5)
ax1.legend(loc='upper left')


ax2 = ax1.twinx()
sns.ecdfplot(
    data=df,
    x='portf_total_amt',
    ax=ax2,
    color='steelblue',
    linewidth=2,
    label='Кумулятивная доля инвесторов'
)
ax2.set_ylabel("Доля инвесторов", fontsize=14)
ax2.legend(loc='lower right')

plt.title("Распределение и кумулятивная доля портфелей инвесторов", fontsize=16)
fig.tight_layout()
plt.show()